# Telnet like server

In [1]:
%discover
%connect test-stm32

test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
test-argon                     serial:///dev/cu.usbmodem1413101        
test-samd                      serial:///dev/cu.usbmodem1413201        
test-stm32                     serial:///dev/cu.usbmodem208E307542521  
test-esp32                     serial:///dev/cu.usbserial-0160B5B8     
Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521


In [1]:
%connect test-stm32
%softreset

import start_urpc
from urpc import *

import sys
sys_ = import_('sys')

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))

Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Platforms:
   host:    pyboard
   server:  esp32


## Server

In [ ]:
import os, io, sys, socket, struct, msgpack, secrets

class Server(io.IOBase):

    def __init__(self, port=1234, buf_size=256):
        self.buffer = bytearray(buf_size)
        self.ss = ss = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ss.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        ss.setblocking(False)

        ai = socket.getaddrinfo('', port)
        addr = ai[0][4]
        ss.bind(addr)
        ss.listen(1)

    def readline(self):
        # self.last = False
        return self._sock.readline().strip().decode()

    def accept(self):
        """Accept connection and serve request.
        Returns immadiately if no client is waiting."""
        try:
            cs, addr = self.ss.accept()
        except Exception as e:
            return
        cs.settimeout(5000)
        self._sock = cs
        print("accepted connection from", addr)
        try:
            cs.write(VERSION)
            cs.write(b'\n')
            pwd = self.readline()
            if pwd != getattr(secrets, 'password', ''):
                cs.write(b'wrong password\n')
                return
            cs.write(b'OK\n')
            self.serve()
        except Exception as e:
            print("*** accept", e)
        finally:
            cs.close()
            print("\nclosed client connection")

    def serve(self):
        # multiple requests or just one?
        while True:
            try:
                # read request header
                req = self.readline()
                if len(req) == 0: continue
                # handle request
                getattr(self, 'do_' + req)()
            except Bye:
                return
            except Exception as e:
                print("Exception in serve:", e)
                sys.print_exception(e)

    # dupterm
    def write(self, data):
        self._sock.write(data)

    def readinto(self, b):
        return None


def start_server(port=54321):
    server = Server(port)

    # advertise
    import network, time
    # network.WLAN(network.STA_IF).mdns_add_service('_mp', '_tcp', port)

    ip = network.WLAN(network.STA_IF).ifconfig()[0]
    print("mp://{}:{}".format(ip, port))

    while True:
        server.accept()
        time.sleep(0.1)
